In [1]:
from splinter import Browser
from bs4 import BeautifulSoup
import requests as rq
import pandas as pd
import time
import re

In [2]:
browser = Browser("chrome", executable_path = "chromedriver", headless=False)

# Visittha NASA mars news site

In [3]:
url='https://mars.nasa.gov/news/'
browser.visit(url)

browser.is_element_present_by_css('ul.item_list li.slide', wait_time=1)

True

In [4]:
html = browser.html
news_soup = BeautifulSoup(html,'html.parser')
slide_elem = news_soup.select_one('ul.item_list li.slide')

In [5]:
slide_elem.find('div',class_='content_title')

<div class="content_title"><a href="/news/8645/mars-helicopter-attached-to-nasas-perseverance-rover/" target="_self">Mars Helicopter Attached to NASA's Perseverance Rover</a></div>

In [6]:
new_title = slide_elem.find('div', class_='content_title').get_text()
new_title

"Mars Helicopter Attached to NASA's Perseverance Rover"

In [7]:
new_p = slide_elem.find('div', class_='article_teaser_body').get_text()
new_p

"The team also fueled the rover's sky crane to get ready for this summer's history-making launch."

# JPL Space Images Featured Image

In [8]:
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)

In [9]:
full_img_elem = browser.find_by_id('full_image')
full_img_elem.click()

In [10]:
browser.is_element_present_by_text('more info', wait_time=1)
more_info_elem = browser.find_link_by_partial_text('more info')
more_info_elem.click()

c:\users\kehoc\anaconda3\envs\pandas\lib\site-packages\splinter\driver\webdriver\__init__.py:528: FutureWarning: browser.find_link_by_partial_text is deprecated. Use browser.links.find_by_partial_text instead.
  FutureWarning,


In [11]:
html = browser.html
img_soup = BeautifulSoup(html, 'html.parser')

In [12]:
img_url_rel = img_soup.select_one('figure.lede a img').get('src')
img_url_rel

'/spaceimages/images/largesize/PIA23170_hires.jpg'

In [13]:
img_url = f'https://www.jpl.nasa.gov{img_url_rel}'
img_url

'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA23170_hires.jpg'

# Mars Weather

In [14]:
url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(url)

time.sleep(5)

In [15]:
html = browser.html
weather_soup = BeautifulSoup(html, 'html.parser')

In [16]:
mars_weather_tweet = weather_soup.find('div', attrs={'class':'tweet','data-name':'Mars Weather'})

In [17]:
try:
    mars_weather = mars_weather_tweet.find('p','tweet-text').get_text()
    mars_weather
    
except AttributeError:
    pattern = re.compile(r'sol')
    mars_weather = weather_soup.find('span', text=pattern).text
    mars_weather
    
mars_weather

'InSight sol 489 (2020-04-11) low -93.5ºC (-136.2ºF) high -6.5ºC (20.3ºF)\nwinds from the WNW at 4.8 m/s (10.7 mph) gusting to 17.4 m/s (39.0 mph)\npressure at 6.60 hPa'

# Mars Hemispheres

In [18]:
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url)

In [19]:
hemisphere_image_urls = []

links = browser.find_by_css('a.product-item h3')

for i in range(len(links)):
    hemisphere = {}
    
    browser.find_by_css('a.product-item h3')[i].click()
    
    sample_elem = browser.find_link_by_text('Sample').first
    hemisphere['img_url'] = sample_elem['href']
    
    hemisphere['title'] = browser.find_by_css('h2.title').text
    
    hemisphere_image_urls.append(hemisphere)
    
    browser.back()

c:\users\kehoc\anaconda3\envs\pandas\lib\site-packages\splinter\driver\webdriver\__init__.py:536: FutureWarning: browser.find_link_by_text is deprecated. Use browser.links.find_by_text instead.
  FutureWarning,


In [20]:
hemisphere_image_urls

[{'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg',
  'title': 'Cerberus Hemisphere Enhanced'},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg',
  'title': 'Schiaparelli Hemisphere Enhanced'},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg',
  'title': 'Syrtis Major Hemisphere Enhanced'},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg',
  'title': 'Valles Marineris Hemisphere Enhanced'}]

# Mars Facts

In [25]:
df = pd.read_html('https://space-facts.com/mars/')[0]

In [26]:
df.columns=['description','value']

In [27]:
df

,description,value
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [28]:
df.to_html('Mars_Facts.png')